In [8]:
# In src/experiments/m2_equities.py
from src.classes.ETL2 import ETL
from src.classes.features import FeatureEngineer  # Assume engineer_features is refactored similarly
import logging 
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
if __name__ == "__main__":
    leading_indicators = ['M2SL']
    equities = ['BTC-USD', 'GLD', 'SLV']
    lag_periods = [1, 3, 6]
    ma_windows = [3, 12]
    horizon = 1
    results = []
    data_dir = "C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/data"

    etl = ETL(data_dir=data_dir)
    feature_engineer = FeatureEngineer(lag_periods=lag_periods, ma_windows=ma_windows)

    for equity in equities:
        logger.info(f"Processing {equity}")
        # Run ETL pipeline
        data = etl.run(stock_list=[equity], indicators=leading_indicators, experiment_name=f"m2_{equity.lower()}")
        logger.info(f"Merged data shape for {equity}: {data.shape}")
        
        # Engineer features
        combined_data = feature_engineer.transform(data, f'{equity}_Close')
        logger.info(f"Engineered data shape for {equity}: {combined_data.shape}")
        
        # Define features
        indicator = "M2SL"
        features = [indicator, f"{indicator}_lag1", f"{indicator}_lag3", f"{indicator}_lag6",
                    f"{equity}_Close_ma3", f"{equity}_Close_ma12", f"{equity}_Volume_lag1", f"{equity}_Close_volatility"]
        
        # Train model (unchanged)
        model, metrics, backtest_df = train_pytorch_forecast(combined_data, f"{equity}_Close", features, horizon)
        
        if backtest_df is not None:
            logger.info(f"Model metrics for {equity}: {metrics}")
            results.append({
                'ticker': equity,
                'horizon': horizon,
                'metrics': metrics,
                'backtest_df': backtest_df
            })

    if results:
        create_dashboard(results)

ERROR:src.classes.data_loaders:Error fetching M2SL: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=M2SL&api_key=17188a6953269ab608ba14c3e3d8fb02-3&realtime_start=2010-01-01&realtime_end=2025-04-30&observation_start=2010-01-01&observation_end=2025-04-30&file_type=json
ERROR:src.classes.ETL2:No data after merging for m2_btc-usd


KeyError: 'BTC-USD_Close'